In [1]:
import shutil
from pathlib import Path

import datasets
import torch as t
from sae_lens import (
    CacheActivationsRunner,
    CacheActivationsRunnerConfig,
)
from sae_lens.store.cached_activation_store import CachedActivationsStore

from crosscoder_lens.config import CrossCoderTrainerConfig
from crosscoder_lens.trainer import CrossCoderTrainingRunner

In [2]:
total_training_steps = 30_000
sae_batch_size = 4096
total_training_tokens = sae_batch_size * total_training_steps
lr_warm_up_steps = 0
lr_decay_steps = total_training_steps // 5  # 20% of training
l1_warm_up_steps = total_training_steps // 20  # 5% of training

d_in = 1024
context_size = 512
dtype = "float32"
device = (
    "cuda"
    if t.cuda.is_available()
    else "mps"
    if t.backends.mps.is_available()
    else "cpu"
)

In [3]:
activations_path = Path("activations")
if activations_path.exists():
    print(f"Removing existing activations directory {activations_path}")
    shutil.rmtree(activations_path)
activations_path.mkdir(exist_ok=False)


def _create_dataset_from_hook(hook_name: str):
    cfg = CacheActivationsRunnerConfig(
        new_cached_activations_path=str(activations_path / hook_name),
        dataset_path="apollo-research/roneneldan-TinyStories-tokenizer-gpt2",
        model_name="tiny-stories-1L-21M",
        hook_name=hook_name,
        hook_layer=0,
        buffer_size_gb=2,
        ### Parameters
        training_tokens=total_training_tokens,
        model_batch_size=256,
        context_size=context_size,
        ###
        d_in=d_in,
        shuffle=False,
        prepend_bos=False,
        device=device,
        dtype=dtype,
    )
    return CacheActivationsRunner(cfg).run()

dataset_pre = _create_dataset_from_hook("blocks.0.hook_resid_pre").rename_column("blocks.0.hook_resid_pre", "pre")
dataset_post = _create_dataset_from_hook("blocks.0.hook_resid_post").rename_column("blocks.0.hook_resid_post", "post")
dataset = datasets.concatenate_datasets([dataset_pre, dataset_post], axis=1)
dataset

Removing existing activations directory activations
Loaded pretrained model tiny-stories-1L-21M into HookedTransformer


Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
cached_activations_store = CachedActivationsStore(
    ds=dataset,
    column_names=["pre", "post"],
    batch_size=sae_batch_size,
    context_size=context_size,
    dl_kwargs={"num_workers": 0, "prefetch_factor": None}, # dataloader doesn't like notebooks
)
print(f"Batch shape: {cached_activations_store.next_batch().shape}")
cached_activations_store.reset_input_dataset()


Batch shape: torch.Size([32, 2, 1024])


In [6]:
trainer_cfg = CrossCoderTrainerConfig(
    n_models=2,
    d_in=d_in,
    expansion_factor=4,
    training_tokens=total_training_tokens,
    normalize_sae_decoder=False,  # has to be False for now
    device=device,
    checkpoint_path="checkpoints",
    ### Copied from SAELens training_a_sparse_autoencoder.ipynb
    lr=5e-5,
    adam_beta1=0.9,
    adam_beta2=0.999,
    lr_scheduler_name="constant",
    lr_warm_up_steps=lr_warm_up_steps,
    lr_decay_steps=lr_decay_steps,
    l1_coefficient=5,
    l1_warm_up_steps=l1_warm_up_steps,
    lp_norm=1.0,
    log_to_wandb=True,
    wandb_project="crosscoder_tiny_stories_1l_21M",
    wandb_log_frequency=30,
    eval_every_n_wandb_logs=20,
)

trainer = CrossCoderTrainingRunner(trainer_cfg, cached_activations_store)
sae = trainer.run()

Run name: 2048-L1-0.001-LR-0.0003-Tokens-5.000e+04
n_tokens_per_buffer (millions): 0.08192
Lower bound: n_contexts_per_buffer (millions): 0.00064
Total training steps: 12
Total wandb updates: 12
n_tokens_per_feature_sampling_window (millions): 1048.576
n_tokens_per_dead_feature_window (millions): 524.288
We will reset the sparsity calculation 0 times.
Number tokens in sparsity calculation window: 8.19e+06
Running training
CrossCoder(
  (activation_fn): ReLU()
  (hook_sae_input): HookPoint()
  (hook_sae_acts_pre): HookPoint()
  (hook_sae_acts_post): HookPoint()
  (hook_sae_output): HookPoint()
  (hook_sae_recons): HookPoint()
  (hook_sae_error): HookPoint()
)


Training SAE:   0%|          | 0/50000 [00:00<?, ?it/s]

n_training_tokens: 32
n_training_tokens: 64
n_training_tokens: 96
n_training_tokens: 128
n_training_tokens: 160
n_training_tokens: 192
n_training_tokens: 224
n_training_tokens: 256
n_training_tokens: 288
n_training_tokens: 320
n_training_tokens: 352
n_training_tokens: 384
n_training_tokens: 416
n_training_tokens: 448
n_training_tokens: 480
n_training_tokens: 512
n_training_tokens: 544
n_training_tokens: 576
n_training_tokens: 608
n_training_tokens: 640
n_training_tokens: 672
n_training_tokens: 704
n_training_tokens: 736
n_training_tokens: 768
n_training_tokens: 800
n_training_tokens: 832
n_training_tokens: 864
n_training_tokens: 896
n_training_tokens: 928
n_training_tokens: 960
n_training_tokens: 992
n_training_tokens: 1024
n_training_tokens: 1056
n_training_tokens: 1088
n_training_tokens: 1120
n_training_tokens: 1152
n_training_tokens: 1184
n_training_tokens: 1216
n_training_tokens: 1248
n_training_tokens: 1280
n_training_tokens: 1312
n_training_tokens: 1344
n_training_tokens: 1376
n_

100| l1_loss: 0.37917 | mse_loss: 165.83426: : 409600it [00:00, 1387046.05it/s]       

n_training_tokens: 2272
n_training_tokens: 2304
n_training_tokens: 2336
n_training_tokens: 2368
n_training_tokens: 2400
n_training_tokens: 2432
n_training_tokens: 2464
n_training_tokens: 2496
n_training_tokens: 2528
n_training_tokens: 2560
n_training_tokens: 2592
n_training_tokens: 2624
n_training_tokens: 2656
n_training_tokens: 2688
n_training_tokens: 2720
n_training_tokens: 2752
n_training_tokens: 2784
n_training_tokens: 2816
n_training_tokens: 2848
n_training_tokens: 2880
n_training_tokens: 2912
n_training_tokens: 2944
n_training_tokens: 2976
n_training_tokens: 3008
n_training_tokens: 3040
n_training_tokens: 3072
n_training_tokens: 3104
n_training_tokens: 3136
n_training_tokens: 3168
n_training_tokens: 3200
n_training_tokens: 3232
n_training_tokens: 3264
n_training_tokens: 3296
n_training_tokens: 3328
n_training_tokens: 3360
n_training_tokens: 3392
n_training_tokens: 3424
n_training_tokens: 3456
n_training_tokens: 3488
n_training_tokens: 3520
n_training_tokens: 3552
n_training_token

...

n_training_tokens: 43552
n_training_tokens: 43584
n_training_tokens: 43616
n_training_tokens: 43648
n_training_tokens: 43680
n_training_tokens: 43712
n_training_tokens: 43744
n_training_tokens: 43776
n_training_tokens: 43808
n_training_tokens: 43840
n_training_tokens: 43872
n_training_tokens: 43904
n_training_tokens: 43936
n_training_tokens: 43968
n_training_tokens: 44000
n_training_tokens: 44032
n_training_tokens: 44064
n_training_tokens: 44096
n_training_tokens: 44128
n_training_tokens: 44160
n_training_tokens: 44192
n_training_tokens: 44224
n_training_tokens: 44256
n_training_tokens: 44288
n_training_tokens: 44320
n_training_tokens: 44352
n_training_tokens: 44384
n_training_tokens: 44416
n_training_tokens: 44448
n_training_tokens: 44480
n_training_tokens: 44512
n_training_tokens: 44544
n_training_tokens: 44576
n_training_tokens: 44608
n_training_tokens: 44640
n_training_tokens: 44672
n_training_tokens: 44704
n_training_tokens: 44736
n_training_tokens: 44768
n_training_tokens: 44800


1500| l1_loss: 0.37315 | mse_loss: 34.35598: : 6144000it [00:04, 1369967.10it/s]

n_training_tokens: 47936
n_training_tokens: 47968
n_training_tokens: 48000
n_training_tokens: 48032
n_training_tokens: 48064
n_training_tokens: 48096
n_training_tokens: 48128
n_training_tokens: 48160
n_training_tokens: 48192
n_training_tokens: 48224
n_training_tokens: 48256
n_training_tokens: 48288
n_training_tokens: 48320
n_training_tokens: 48352
n_training_tokens: 48384
n_training_tokens: 48416
n_training_tokens: 48448
n_training_tokens: 48480
n_training_tokens: 48512
n_training_tokens: 48544
n_training_tokens: 48576
n_training_tokens: 48608
n_training_tokens: 48640
n_training_tokens: 48672
n_training_tokens: 48704
n_training_tokens: 48736
n_training_tokens: 48768
n_training_tokens: 48800
n_training_tokens: 48832
n_training_tokens: 48864
n_training_tokens: 48896
n_training_tokens: 48928
n_training_tokens: 48960
n_training_tokens: 48992
n_training_tokens: 49024
n_training_tokens: 49056
n_training_tokens: 49088
n_training_tokens: 49120
n_training_tokens: 49152
n_training_tokens: 49184


1500| l1_loss: 0.37315 | mse_loss: 34.35598: : 6144000it [00:05, 1201630.96it/s]


details/current_l1_coefficient,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
details/current_learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
details/n_training_tokens,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇████
losses/l1_loss,█▆▄▃▂▂▂▂▂▂▁▂▁▂▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▂▂▃▃
losses/mse_loss,█▇▆▇▇▅▅▄▃▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁
losses/overall_loss,█▅▄▄▃▃▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/raw_l1_loss,██▆▆▄▅▄▁▂▂▂▁▂▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▄▅▄▅▅▅▅▅▅▅▅
metrics/explained_variance,▁▆▇█▇███████████████████████████████████
metrics/explained_variance_std,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/l0,█▅▄▄▄▃▂▂▂▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄
sparsity/dead_features,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
